## Generalized Policy Iteration 
We write two algorithms as functions to be used with generic MDPs. The first of these is Policy iteration, and the second is Value iteration. Both of these methods are instances of Generalized Policy Iteration algorithms, where we evaluate a policy, then make the policy greedy with respect to the value function. 

## Policy Iteration: 
Here, we perform policy evaluation and improvement one after the other till we get a policy that is stable

In [75]:
def policy_iteration(states, action_space, reward_per_state, gamma, transition_probabilities): 
    theta = 1e-4
    V = [0 for i in range(len(states))] #initialize Value function
    pi = [0 for i in range(len(states))]
    max_iterations = 10000
    for i in range(max_iterations): 
        
        #perform policy evaluation 
        for j in range(max_iterations): 
            max_diff = 0 
            
            for state in states: 
                val = 0
                for next_state in states: 
                    prob = transition_probabilities[state][next_state][pi[state]]
                    next_state_reward = reward_per_state[next_state]
                    val += prob * (gamma * V[next_state] + next_state_reward)
                
                max_diff = max(max_diff, abs(V[state]-val))
                V[state] = val
            
            if max_diff < theta: 
                break 
        #perform policy improvement 
        optimal_policy = True 
        for state in states: 
            current_value = V[state]
            for action in action_space: 
                val = 0
                for next_state in states: 
                    prob = transition_probabilities[state][next_state][action]
                    val += prob * (gamma * V[next_state] + reward_per_state[next_state])
                
                if val > current_value: 
                    pi[state] = action 
                    optimal_policy = False
        
        if optimal_policy: 
            break 

    return V, pi 

### Value Iteration
This method is more efficient (in most cases) when compared to the prior because we perform a singular backup during the evaluation phase. Here, we simply pick the action that maximizes our value function with one backup step. This can be computationally cheaper to perform than the algorithm above. 
Please Note, in this case we only build at deterministic policies. More often than not, we're going to want to build stochastic policies instead. 

In [76]:
def value_iteration(action_space, states, reward_per_state, gamma, transition_probabilities): 
    max_iterations = 10000 
    theta = 1e-4
    V = [0 for i in range(len(states))] #initialize Value function
    pi = [None for i in range(len(states))] #Initialize policy 
    
    #run until change in value is smaller than theta or max iterations counted 
    for i in range(max_iterations): 
        max_diff = 0 
        for state in states: 
            old_val = V[state]
            max_val = 0 
            for action in action_space: 
                val = reward_per_state[state] 
                #calculate discounted return 
                for next_state in states: 
                    prob = transition_probabilities[state][next_state][action]
                    val += prob * (gamma * V[next_state] + reward_per_state[next_state])
                if val > max_val:   
                    max_val = val
                #pick action that maximizes value 
                if V[state] < val: 
                    pi[state] = action_space[action]
                    V[state] = val
            
            max_diff = max(max_diff, abs(V[state]-old_val))

        if max_diff < theta: 
            break 
    
    return V, pi 

### Evaluating both GPI methods over different MDPs 
Over the next section we'll build some basic Markov Decision Processes and apply these two iterative methods to them. We can then compare runtime and convergence of the two. 

1. A boxing MDP. States are as follows: wide stanced (0), narrow stanced (1), reverse stanced (2)
Actions are: widen stance, narrow stance, switch stance, throw punch 
when we widen our stance, we end up in wide stance (0), when we narrow we end up in narrow and when we reverse we end up in 2. When we throw a punch we can end up in a wide stance or a narrow stance with 50% probability. 

Note: The above defined MDP does not make any real life sense, but was a quick experiment to come up with an MDP formulation to play around with. during the process of learning, I tweaked all of the numbers in the rewards and transition probability matrices below to see how the optimal policy selected evolves. 

In [77]:
states = [0, 1, 2]
actions = [0, 1, 2, 3]

rewards = [5, -5, -9]


In [78]:

transition_probs = [[[0.6, 0, 0, 0.4], [0, .8, 0.1,0.1], [0, 0, 1, 0]], 
                    [[1, 0, 0, 0], [0, 0.7, 0, 0.3], [0, 0, 0.9, 0.1]], 
                    [[1, 0, 0, 0], [0,0,1,0], [0, 0, 0,1]]]

gamma = 0.5 

value_iteration(action_space=actions, states=states, reward_per_state=rewards, gamma=gamma, transition_probabilities=transition_probs)

([11.42855177369, 5.714275886845, 1.7142758868449999], [0, 0, 0])

In [79]:
states = [0, 1, 2]
actions = [0, 1, 2, 3]

rewards = [5, -5, -9]

transition_probs = [[[0.6, 0, 0, 0.4], [0, .8, 0.1,0.1], [0, 0, 1, 0]], 
                    [[1, 0, 0, 0], [0, 0.7, 0, 0.3], [0, 0, 0.9, 0.1]], 
                    [[1, 0, 0, 0], [0,0,1,0], [0, 0, 0,1]]]

gamma = 0.5 

value_iteration(action_space=actions, states=states, reward_per_state=rewards, gamma=gamma, transition_probabilities=transition_probs)

([11.42855177369, 5.714275886845, 1.7142758868449999], [0, 0, 0])

In [80]:
transition_probs[0]

[[0.6, 0, 0, 0.4], [0, 0.8, 0.1, 0.1], [0, 0, 1, 0]]

In [81]:
policy_iteration(states, actions, rewards, gamma, transition_probs)

([4.285714285714286, 7.142857142857142, 7.142857142857142], [0, 0, 0])